# Imports and packages

In [1]:
## helpful packages

import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import random
import re
import string

## nltk imports
import nltk
### uncomment and run these lines if you haven't downloaded relevant nltk add-ons yet
#nltk.download('averaged_perceptron_tagger')
#nltk.download('stopwords')
from nltk import pos_tag
from nltk.tokenize import word_tokenize, wordpunct_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords

## spacy imports
import spacy
### uncomment and run the below line if you haven't loaded the en_core_web_sm library yet
#! python3 -m spacy download en_core_web_sm
import en_core_web_sm
nlp = en_core_web_sm.load()

## vectorizer
from sklearn.feature_extraction.text import CountVectorizer

## sentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

## lda
from gensim import corpora
import gensim

## repeated printouts and wide-format text
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pd.set_option('display.max_colwidth', None)

# Text analysis of Department of Justice (DOJ) press releases (50 points total)

- For background:

    - DOJ is the federal law enforcement agency responsible for federal prosecutions; this contrasts with the local prosecutions in the Cook County dataset we analyzed earlier. Here's a short explainer on which crimes get prosecuted federally versus locally: https://www.criminaldefenselawyer.com/resources/criminal-defense/federal-crime/state-vs-federal-crimes.htm#:~:text=Federal%20criminal%20prosecutions%20are%20handled,of%20state%20and%20local%20law. 
    - Here's the Kaggle that contains the data: https://www.kaggle.com/jbencina/department-of-justice-20092018-press-releases 
    - Here's the code the dataset creator used to scrape those press releases here if you're interested: https://github.com/jbencina/dojreleases
    
- See here for a codebook: https://docs.google.com/spreadsheets/d/1UopmSvFGrwJvz_c3Plh32Yxkqwff64oS_CcpfATOV8k/edit?usp=sharing

In [2]:
## first, unzip the combined.json.zip file
## then, run this code to load the unzipped json file and convert to a dataframe
## and convert some of the attributes from lists to values
## make sure to change the pathname if you need to
doj = pd.read_json("combined.json", lines = True)

## due to json, topics are in a list so remove them and concatenate with ;
doj['topics_clean'] = ["; ".join(topic) 
                      if len(topic) > 0 else "No topic" 
                      for topic in doj.topics]

## similarly with components
doj['components_clean'] = ["; ".join(comp) 
                           if len(comp) > 0 else "No component" 
                           for comp in doj.components]

## drop older columns from data
doj = doj[['id', 'title', 'contents', 'date', 'topics_clean', 
           'components_clean']].copy()

doj.head()


id  \
0     None   
1  12-919    
2  11-1002   
3   10-015   
4   18-898   

                                                                                                          title  \
0                                                                  Convicted Bomb Plotter Sentenced to 30 Years   
1                              $1 Million in Restitution Payments Announced to Preserve North Carolina Wetlands   
2                 $1 Million Settlement Reached for Natural Resource Damages at Superfund Site in Massachusetts   
3                                          10 Las Vegas Men Indicted \r\nfor Falsifying Vehicle Emissions Tests   
4  $100 Million Settlement Will Speed Cleanup Work at Centredale Manor Superfund Site in North Providence, R.I.   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

## 1. Tagging and sentiment scoring (16 points)

Focus on the following press release: `id` == "17-1204" about this pharmaceutical kickback prosecution: https://www.forbes.com/sites/michelatindera/2017/11/16/fentanyl-billionaire-john-kapoor-to-plead-not-guilty-in-opioid-kickback-case/?sh=21b8574d6c6c 

The `contents` column is the one we're treating as a document. You may need to to convert it from a pandas series to a single string.

We'll call the raw string of this press release `pharma`

In [3]:
## your code to subset to one press release and take the string
pharma = doj[doj.id == "17-1204"].contents.to_string()
pharma

'4909    The founder and majority owner of Insys Therapeutics Inc., was arrested today and charged with leading a nationwide conspiracy to profit by using bribes and fraud to cause the illegal distribution of a Fentanyl spray intended for cancer patients experiencing breakthrough pain.\xa0"More than 20,000 Americans died of synthetic opioid overdoses last year, and millions are addicted to opioids. And yet some medical professionals would rather take advantage of the addicts than try to help them," said Attorney General Jeff Sessions. "This Justice Department will not tolerate this.\xa0 We will hold accountable anyone – from street dealers to corporate executives -- who illegally contributes to this nationwide epidemic.\xa0 And under the leadership of President Trump, we are fully committed to defeating this threat to the American people.”John N. Kapoor, 74, of Phoenix, Ariz., a current member of the Board of Directors of Insys, was arrested this morning in Arizona and charged with RIC

### 1.1 part of speech tagging (3 points)

A. Preprocess the `pharma` press release to remove all punctuation / digits (so can use `.isalpha()` to subset)

B. With the preprocessed press release from part A, use the part of speech tagger within nltk to tag all the words in that one press release with their part of speech. 

C. Using the output from B, extract the adjectives and sort those adjectives from most occurrences to fewest occurrences. Print a dataframe with the 5 most frequent adjectives and their counts in the `pharma` release. See here for a list of the names of adjectives within nltk: https://pythonprogramming.net/natural-language-toolkit-nltk-part-speech-tagging/

**Resources**:

- Documentation for .isalpha(): https://www.w3schools.com/python/ref_string_isalpha.asp
- `process_step1` function here has an example of tokenizing and filtering to words where .isalpha() is true: 
https://github.com/rebeccajohnson88/PPOL564_slides_activities/blob/main/activities/fall_22/solutions/09_textasdata_partII_topicmodeling_solution.ipynb 
- Part of speech tagging section of this code: 
https://github.com/rebeccajohnson88/PPOL564_slides_activities/blob/main/activities/fall_22/solutions/08_textasdata_partI_textmining_solutions.ipynb



In [4]:
## your code here to restrict to alpha
# First, we slice the text to words, so we can analyze the texts
word_pharma= word_tokenize(pharma)
#Then, we use is.alpha to remove all punctuation and digits
list_word_pharma = [word for word in word_pharma if word.isalpha()]

In [5]:
## your code here for part of speech tagging
#Tag the part of speech
tag_pharma = pos_tag(list_word_pharma)
#I extract the adjectives and put 1 as an occurance
adj_pharma = pd.DataFrame({"words": [pos[0] for pos in tag_pharma if "JJ" in pos[1]], 
                          "occurance": 1})
#Then, I group the same words and add the occurance values to find out how many times it appeared
#I printed the top 5 most often found adjectives
##what does [:5] exactly mean?
adj_pharma.groupby("words").sum().sort_values('occurance', ascending = False)[:5]

,occurance
words,
former,8
opioid,5
nationwide,4
other,3
addictive,3


## 1.2 named entity recognition (3 points)



A. Using the original `pharma` press release (so the one before stripping punctuation/digits), use spaCy to extract all named entities from the press release.

B. Print the unique named entities with the tag: `LAW`. Here's some background on what RICO means: https://www.justia.com/criminal/docs/rico/ 

**Resources**:
- For parts A and B: named entity recognition part of this code: 
https://github.com/rebeccajohnson88/PPOL564_slides_activities/blob/main/activities/fall_22/solutions/08_textasdata_partI_textmining_solutions.ipynb

In [6]:
## your code here 
#A: Using spaCy to extract al name entities
#convert to spaCy token
spacy_pharma = nlp(pharma)
#The entity of the spacy_pharma
entities_pharma = spacy_pharma.ents

#B: print the unique named entities with the tag "LAW"
set([entity.text for entity in entities_pharma if entity.label_ == "LAW"])


{'RICO', 'the Controlled Substances Act'}

C. You want to extract the possible sentence lengths the CEO is facing; pull out the named entities with (1) the label `DATE` and (2) that contain the word year or years. Print these named entities.

**Hint:**  
You may want to use the `re` module for the second part.

In [7]:
## your code here
#C: named entities with the label "DATE"
list_ent = [entity.text for entity in entities_pharma if (entity.label_ == "DATE")]
list_ent = [word for word in list_ent if "year" in word]
list_ent


['last year', 'three years', 'three years']

D. Parse the pharma string at the sentence level. Note that this involves more than just splitting on each `.`; for full credit, add at least one additional delimiter that marks the end of the sentence.

Then, using those sentences, pull and print the original sentences from the press releases where those year lengths are mentioned. Describe in your own words (1 sentence) what length of sentence (prison) and probation (supervised release) the CEO may be facing if convicted after this indictment (if there are multiple lengths mentioned describe the maximum). 

**Hint:**  
You may want to use re.search or re.findall 

**Resources**:

- re.search and re.findall examples here for filtering to ones containing year (multiple approaches; some need not involve `re`): 
https://github.com/rebeccajohnson88/PPOL564_slides_activities/blob/main/activities/fall_22/solutions/07_regex_solutions.ipynb


In [8]:
## your code here
pattern = r"\.\xa0|\."
pharma_sentence = re.split(pattern, pharma)

#Then, using those sentences, pull and print the original sentences 

pattern2 = "|".join(list_ent)
[sentence for sentence in pharma_sentence if re.findall(pattern2, sentence)]

#no greater than 20 years in prison, three years of supervised release
#a sentence of no greater than five years in prison, three years of supervised release


#from the press releases where those year lengths are mentioned. 
#Describe in your own words (1 sentence) what length of sentence (prison) and probation (supervised release) 
#the CEO may be facing if convicted after this indictment 
#(if there are multiple lengths mentioned describe the maximum).
#do the next part of the problem

['"More than 20,000 Americans died of synthetic opioid overdoses last year, and millions are addicted to opioids',
 'The charges of conspiracy to commit RICO and conspiracy to commit mail and wire fraud each provide for a sentence of no greater than 20 years in prison, three years of supervised release and a fine of $250,000, or twice the amount of pecuniary gain or loss',
 ' The charges of conspiracy to violate the Anti-Kickback Law provide for a sentence of no greater than five years in prison, three years of supervised release and a $25,000 fine']

## 1.3 sentiment analysis  (10 points)

- Sentiment analysis section of this script: 
https://github.com/rebeccajohnson88/PPOL564_slides_activities/blob/main/activities/fall_22/solutions/08_textasdata_partI_textmining_solutions.ipynb


A. Subset the press releases to those labeled with one of three topics via `topics_clean`: Civil Rights, Hate Crimes, and Project Safe Childhood. We'll call this `doj_subset` going forward and it should have 717 rows.



In [9]:
## your code here for subsetting
doj_subset = doj[doj["topics_clean"].isin(["Civil Rights",'Hate Crimes',"Project Safe Childhood"])].reset_index()
doj_subset

,index,id,title,contents,date,topics_clean,components_clean
0,77,17-1235,Additional Former Correctional Officer Pleads Guilty to Beating of Handcuffed and Shackled Inmate at Angola State Prison,"A former supervisory correctional officer at Louisiana State Penitentiary in Angola, Louisiana, pleaded guilty yesterday in connection with the beating of a handcuffed and shackled inmate, in addition to conspiring to cover up their misconduct by falsifying official records and lying to internal investigators about what happened. James Savoy, 39, of Marksville, Louisiana, admitted during his plea hearing that he witnessed other officers using excessive force against the inmate and failed to intervene; that he conspired with other officers to cover up the beating by engaging in a variety of obstructive acts; and that he personally falsified official prison records to cover up the attack. Scotty Kennedy, 48, of Beebe, Arkansas, and John Sanders, 30, of Marksville, Louisiana previously pleaded guilty in November 2016, and September 2017, for their roles in the beating and cover up. “Every citizen has the right to due process and protection from unreasonable force, and correctional officers who violate these basic Constitutional rights must be held accountable for their egregious actions” said Acting Assistant Attorney General John Gore of the Civil Rights Division. “The Justice Department will continue to vigorously prosecute correctional officers who violate the public’s trust by committing crimes and to covering up violations of federal criminal law.” “Yesterday is another example of our office’s unwavering commitment to pursuing those who violate the federal criminal civil rights laws,” said Acting United States Attorney for the Middle District of Louisiana Corey Amundson. “We will continue to work closely with the Justice Department’s Civil Rights Division and the FBI to ensure that no one is above the law.” This case is being investigated by the FBI’s Baton Rouge Resident Agency and is being prosecuted by Assistant U.S. Attorney Frederick A. Menner, Jr. of the Middle District of Louisiana and Trial Attorney Christopher J. Perras of the Civil Rights Division’s Criminal Section.",2017-11-02T00:00:00-04:00,Civil Rights,"Civil Rights Division; USAO - Louisiana, Middle"
1,155,15-1522,Alabama Man Found Guilty of Aggravated Sexual Abuse of a Child,"A federal jury convicted Rick Lee Evans, 43, of Anniston, Alabama, today of aggravated sexual abuse of a child after a five-day trial, Assistant Attorney General Leslie R. Caldwell of the Justice Department’s Criminal Division and U.S. Attorney Joyce White Vance of the Northern District of Alabama announced. According to evidence introduced at trial, Evans, a former U.S. Army soldier, and his then-wife, a Department of Defense employee, were residing in Germany when they were asked to take temporary custody of a five-year-old child whose parents were deployed to Iraq with the U.S. Army. Evans sexually abused the child on multiple occasions during the 18 months that the child lived with him from May 2007 to December 2008. Trial Attorney Austin M. Berry of the Criminal Division’s Child Exploitation and Obscenity Section (CEOS) and Assistant U.S. Attorney Jacquelyn Hutzell of the Northern District of Alabama are prosecuting the case. U.S. Army Criminal Investigations Division and the FBI’s Birmingham, Alabama, Division investigated the case. This case was brought as part of Project Safe Childhood, a nationwide initiative to combat the growing epidemic of child sexual exploitation and abuse, launched in May 2006 by the Department of Justice. Led by U.S. Attorneys’ offices and CEOS, Project Safe Childhood marshals federal, state and local resources to better locate, apprehend and prosecute individuals who exploit children via the Internet, as well as to identify and rescue victims. For more information about Project Safe Childhood, please visit www.justice.gov/psc.",2015-12-11T00:00:00-05:00,Project S

B. Write a function that takes one press release string as an input and:

- Removes named entities from each press release string (**Hint:** you may want to use `re.sub` with an or condition)
- Scores the sentiment of the entire press release using the `SentimentIntensityAnalyzer` and `polarity_scores`
- Returns the length-four (negative, positive, neutral, compound) sentiment dictionary (any order is fine)

Apply that function to each of the press releases in `doj_subset`. 

**Hints**: 

- I used a function + list comprehension to execute and it takes about 30 seconds on my local machine; if it's taking a very long time, you may want to check your code for inefficiencies. If you can't fix those, for partial credit on this part/full credit on remainder, you can take a small random sample of the 717


In [10]:
# your code here defining the function
def sentiment_search(press): 
    #Removes named entities from each press release string 
    #1) converting to the spacy token and then to string list
    
    spacy_press= nlp(press)
    spacy_list = [entity.text for entity in spacy_press.ents]
    
    #2) remove the entities of the token 
    pattern = "|".join([re.escape(entity.text) for entity in spacy_press.ents])
    clean_press = re.sub(pattern, "", press)

    #Scores the sentiment of the entire press release using the SentimentIntensityAnalyzer and polarity_scores
    sent_obj = SentimentIntensityAnalyzer()
    sent_score = sent_obj.polarity_scores(clean_press)
    
    #Returns the length-four (negative, positive, neutral, compound) sentiment dictionary (any order is fine)
    return sent_score

In [11]:
# your code here executing the function
[sentiment_search(press) for press in doj_subset.contents]

[{'neg': 0.199, 'neu': 0.752, 'pos': 0.049, 'compound': -0.9931},
 {'neg': 0.134, 'neu': 0.798, 'pos': 0.069, 'compound': -0.9325},
 {'neg': 0.091, 'neu': 0.834, 'pos': 0.075, 'compound': -0.7579},
 {'neg': 0.126, 'neu': 0.789, 'pos': 0.085, 'compound': -0.9037},
 {'neg': 0.18, 'neu': 0.776, 'pos': 0.045, 'compound': -0.9864},
 {'neg': 0.149, 'neu': 0.798, 'pos': 0.053, 'compound': -0.987},
 {'neg': 0.162, 'neu': 0.759, 'pos': 0.079, 'compound': -0.9595},
 {'neg': 0.088, 'neu': 0.84, 'pos': 0.071, 'compound': -0.6597},
 {'neg': 0.106, 'neu': 0.833, 'pos': 0.061, 'compound': -0.9136},
 {'neg': 0.162, 'neu': 0.783, 'pos': 0.055, 'compound': -0.9801},
 {'neg': 0.217, 'neu': 0.74, 'pos': 0.043, 'compound': -0.9972},
 {'neg': 0.095, 'neu': 0.841, 'pos': 0.064, 'compound': -0.8519},
 {'neg': 0.083, 'neu': 0.852, 'pos': 0.066, 'compound': -0.6486},
 {'neg': 0.33, 'neu': 0.631, 'pos': 0.039, 'compound': -0.995},
 {'neg': 0.182, 'neu': 0.759, 'pos': 0.059, 'compound': -0.9907},
 {'neg': 0.123, 

C. Add the four sentiment scores to the `doj_subset` dataframe to create a dataframe: `doj_subset_wscore`. Sort from highest neg to lowest neg score (so most negative to least negative) and print the `id`, `contents`, and `neg` columns of the two most negative press releases. 

Notes:

- Don't worry if your sentiment score differs slightly from our output on GitHub; differences in preprocessing can lead to diff scores

In [12]:
# you recode here
def with_score(index):
    org = pd.DataFrame(doj_subset.iloc[index]).transpose()
    new = pd.DataFrame(sentiment_search(doj_subset.iloc[index].contents), index =[index])
    return pd.concat([org, new], axis = 1)
    
#dataframe that has four sentiment scores
doj_subset_wscore = pd.concat([with_score(index) for index in range(doj_subset.shape[0])])
doj_subset_wscore.sort_values(by = "neg", ascending = False)[["id", 'contents', 'neg']].iloc[[0,1]]

,id,contents,neg
13,14-248,"The Department of Justice announced that this morning John W. Ng, 58, of Albuquerque, N.M., made his initial appearance in federal court on a criminal complaint charging him with a hate crime offense. This charge is related to anti-Semitic threats Ng made against a Jewish woman who owns and operates the Nosh Jewish Delicatessen and Bakery in Albuquerque. Ng was arrested by the FBI on March 7, 2014, based on a criminal complaint alleging that he interfered with the victim’s federally protected rights by threatening her and interfering with her business because of her religion. According to the criminal complaint, between Jan. 22, 2014, and Feb. 8, 2014, Ng allegedly posted threatening anti-Semitic notes on and in the vicinity of the victim’s business. A criminal complaint merely establishes probable cause, and Ng is presumed innocent unless proven guilty. If convicted on the offense charged in the criminal complaint, Ng faces a maximum statutory penalty of one year in prison. This matter was investigated by the Albuquerque Division of the FBI and is being prosecuted by Assistant U.S. Attorney Mark T. Baker of the U.S. Attorney’s Office for the District of New Mexico and Trial Attorney AeJean Cha of the U.S. Department of Justice’s Civil Rights Division.",0.330
34,13-312,"John Hall, 27, an Aryan Brotherhood member and inmate at the Federal Correctional Institution (FCI) in Seagoville, Texas, was sentenced today by U.S. District Judge Reed O’Connor after pleading guilty to violating the Matthew Shepard and James Byrd Jr. Hate Crimes Prevention Act stemming from his assault of a fellow inmate, whom he believed to be gay, the Department of Justice announced. Hall assaulted his fellow inmate with a dangerous weapon, causing bodily injury to the victim on Dec. 20, 2011. Hall was sentenced to serve 71 months in prison to be served consecutively with the sentence he is currently serving. The assault occurred on Dec. 20, 2011, inside the FCI Seagoville when Hall targeted and attacked the victim, a fellow inmate, because he believed the victim was gay or involved in a sexual relationship with another male inmate. Hall repeatedly punched, kicked and stomped on the victim’s face with his shod feet, a dangerous weapon, while yelling a homophobic slur. The victim lost consciousness during the assault and suffered multiple lacerations to his face. The victim also sustained a fractured eye socket, lost a tooth, fractured other teeth and was treated at a hospital for the injuries he sustained during Hall’s unprovoked attack. Hall pleaded guilty to violating the Matthew Shepard and James Byrd Jr. Hate Crimes Prevention Act on Nov. 8, 2012. “Brutality and violence based on sexual orientation has no place in a civilized society,” said Thomas E. Perez, Assistant Attorney General for the Civil Rights Division. “The Justice Department is committed to using all the tools in our law enforcement arsenal, including the Matthew Shepard and James Byrd Jr. Hate Crimes Prevention Act, to prosecute acts motivated by hate.” “This prosecution sends a clear message that this office, in partnership with attorneys in the department’s Civil Rights Division, will prioritize and aggressively prosecute hate crimes and others civil rights violations in North Texas,” said U.S. Attorney Sarah R. Saldaña of the Northern District of Texas. This case was investigated by the FBI Dallas Division. The case was prosecuted by Assistant U.S. Attorney Errin Martin and Trial Attorney Adriana Vieco of the Civil Rights Division.",0.302


D. With the dataframe from part C, find the mean compound sentiment score for each of the three topics in `topics_clean` using group_by and agg.

E. Add a 1 sentence interpretation of why we might see the variation in scores (remember that compound is a standardized summary where -1 is most negative; +1 is most positive)


In [13]:
## agg and find the mean compound score by topic

doj_subset_wscore.groupby("topics_clean").agg({"compound": ['mean']})

,compound
,mean
topics_clean,
Civil Rights,-0.093421
Hate Crimes,-0.935567
Project Safe Childhood,-0.669722


(Source: https://www.justice.gov/psc)
Both Hate Crimes and Project Safe Childhood target atrocious acts in community that we should prevent. For instance, Project Safe Childhood is an initiative to protect children from sexual exploitation. Thus, I think it is reasonable for the press release to use negative words to recognize the brutality of the events.

# 2. Topic modeling (25 points)

For this question, use the `doj_subset_wscores` data that is restricted to civil rights, hate crimes, and project safe childhood and with the sentiment scores added


## 2.1 Preprocess the data by removing stopwords, punctuation, and non-alpha words (5 points)

A. Write a function that:

- Takes in a single raw string in the `contents` column from that dataframe
- Does the following preprocessing steps:

    - Converts the words to lowercase
    - Removes stopwords, adding the custom stopwords in the code cell below to the default stopwords list
    - Only retains alpha words (so removes digits and punctuation)
    - Only retains words 4 characters or longer
    - Uses the snowball stemmer from nltk to stem

- Returns a joined preprocessed string (so if press release is something like "The CEO was indicted" it might return "ceo indict" 
    
B. Use `apply` or list comprehension to execute that function and create a new column in the data called `processed_text`. Note: there will be a deduction if your code uses a non-list comprehension for loop that uses append.
    
C. Print the `id`, `contents`, and `processed_text` columns for the following press releases:

id = 16-718 (this case: https://www.seattletimes.com/nation-world/doj-miami-police-reach-settlement-in-civil-rights-case/)

id = 16-217 (this case: https://www.wlbt.com/story/32275512/three-mississippi-correctional-officers-indicted-for-inmate-assault-and-cover-up/)
    
**Resources**:

- Here's code examples for the snowball stemmer: https://www.geeksforgeeks.org/snowball-stemmer-nlp/
- Here's code with topic modeling steps: 
https://github.com/rebeccajohnson88/PPOL564_slides_activities/blob/main/activities/fall_22/solutions/09_textasdata_partII_topicmodeling_solution.ipynb

In [14]:
custom_doj_stopwords = ["civil", "rights", "division", "department", "justice",
                        "office", "attorney", "district", "case", "investigation", "assistant",
                       "trial", "assistance", "assist"]

In [15]:
## your code defining a text processing function
def transforming (text):
    #Converts the words to lowercase
    lower_text = text.lower()
    
    #Split by words before removing stopwords
    split_text = lower_text.split(" ")
    
    #bringing in the stopwords
    list_stopwords = stopwords.words("english")
    
    #adding the custom stopwords
    list_stopword_add = list_stopwords+custom_doj_stopwords
    
    #Removes stopwords
    remove_text = [word for word in split_text if word not in list_stopword_add]
    
    #Only retains alpha words (so removes digits and punctuation)
    alpha_text = [word for word in remove_text if word.isalpha()]
    
    #Only retains words 4 characters or longer
    long_text = [word for word in alpha_text if len(word)>=4]

    #Uses the snowball stemmer from nltk to stem
    snow_stemmer = SnowballStemmer(language= 'english')
    stem_text = [snow_stemmer.stem(word) for word in long_text]

    #Returns a joined preprocessed string
    return " ".join(stem_text)
    

In [16]:
## your code executing the function
#B. Use apply or list comprehension to execute that function and create a new column in the data called processed_text. 

processed = pd.DataFrame([transforming(doj_subset_wscore.contents[index]) for index in range(doj_subset_wscore.shape[0])], columns = ["processed_text"])
processed
doj_subset_wscores_proc= pd.concat([doj_subset_wscore, processed], axis = 1)


,processed_text
0,former supervisori correct offic louisiana state penitentiari plead guilti yesterday connect beat handcuf shackl addit conspir cover misconduct falsifi offici record lie intern investig jame admit plea hear wit offic use excess forc inmat fail conspir offic cover beat engag varieti obstruct person falsifi offici prison record cover scotti john louisiana previous plead guilti novemb septemb role beat cover citizen right process protect unreason correct offic violat basic constitut must held account egregi said act general john gore continu vigor prosecut correct offic violat trust commit crime cover violat feder crimin anoth exampl unwav commit pursu violat feder crimin said act unit state middl louisiana corey continu work close ensur investig baton roug resid agenc prosecut frederick middl louisiana christoph perra crimin
1,feder juri convict rick today aggrav sexual abus child general lesli caldwel crimin joyc white vanc northern alabama accord evid introduc former armi defens resid germani ask take temporari custodi child whose parent deploy iraq evan sexual abus child multipl occas month child live decemb austin berri crimin child exploit obscen section jacquelyn hutzel northern alabama prosecut armi crimin investig investig brought part project safe nationwid initi combat grow epidem child sexual exploit launch offic project safe childhood marshal state local resourc better apprehend prosecut individu exploit children well identifi rescu inform project safe pleas visit
2,alabama nativ indict today charg multipl crime involv travel intent engag illicit sexual conduct minor child announc general lesli caldwel crimin kenyen brown southern clarenc edward ever technolog teacher employ conecuh board arrest charg today five count travel intent engag illicit sexual conduct count attempt travel intent engag illicit sexual conduct count product attempt product child count transport child count receipt child count access intent view child pornographi count possess child accord ever alleg travel thailand summer purpos engag illicit sexual conduct minor alleg attempt make similar trip spring ever also alleg photograph abus transport imag back unit ever alleg imag child sexual exploit comput electron charg contain indict ever presum innoc unless proven guilti beyond reason doubt court investig jame burk crimin child exploit obscen section attorney sean costello maria murphi southern alabama prosecut brought part project safe nationwid initi combat grow epidem child sexual exploit abus launch offic project safe childhood marshal state local resourc better apprehend prosecut individu exploit children well identifi rescu inform project safe pleas visit
3,alabama indict today feder grand juri charg relat product child pornographi involv four minor announc general lesli caldwel crimin joyc white vanc northern gregori jerom former cullman indict four count product child count conspiraci advertis child pornographi count conspiraci distribut receiv child accord septemb decemb coerc entic minor engag sexual explicit conduct order produc imag septemb august conspir individu distribut receiv child pornographi varieti includ postal inspect servic investig larson crimin child exploit obscen section jacquelyn hutzel northern alabama prosecut charg alleg contain indict mere defend presum innoc unless proven member public inform relat matter call uspi birmingham brought part project safe nationwid initi combat grow epidem child sexual exploit abus launch offic project safe childhood marshal state local resourc better apprehend prosecut individu exploit children well identifi rescu inform project safe pleas visit
4,jeremi heath higgin indict threaten threaten anoth person order higgin leav restaur act general jocelyn samuel joyc vanc northern alabama announc charg three count indict return yesterday feder grand juri court northern indict charg feloni count misdemeanor count interfer indict alleg june higgin approach threaten alabama r

In [17]:
## your code showing the examples
doj_subset_wscores_proc[['id', 'contents', 'processed_text']][(doj_subset_wscores_proc.id == "16-718")|(doj_subset_wscores_proc.id == "16-217")]


,id,contents,processed_text
313,16-217,"The Justice Department has reached a comprehensive settlement agreement with the city of Miami and the Miami Police Department (MPD) resolving the Justice Department’s investigation of officer-involved shootings by MPD officers, announced Principal Deputy Assistant Attorney General Vanita Gupta, head of the Justice Department’s Civil Rights Division and U.S. Attorney Wifredo A. Ferrer of the Southern District of Florida. The settlement, which was approved by Miami’s city commission today and will go into effect when the agreement is signed by all parties, resolves claims stemming from the Justice Department’s investigation into officer-involved shootings by MPD officers, which was conducted under the Violent Crime Control and Law Enforcement Act of 1994. The investigation’s findings, issued in July 2013, identified a pattern or practice of excessive use of force through officer-involved shootings in violation of the Fourth Amendment of the Constitution. The city’s compliance with the settlement will be monitored by an independent reviewer, former Tampa, Florida, Police Chief Jane Castor. Under the settlement agreement, the city will implement comprehensive reforms to ensure constitutional policing and support public trust. The settlement agreement is designed to minimize officer-involved shootings and to more effectively and quickly investigate officer-involved shootings that do occur, through measures that include: “This settlement represents a renewed commitment by the city of Miami and Chief Rodolfo Llanes to provide constitutional policing for Miami residents and to protect public safety through sustainable reform,” said Principal Deputy Assistant Attorney General Gupta. “The agreement will help to strengthen the relationship between the MPD and the communities they serve by improving accountability for officers who fire their weapons unlawfully, and provides for community participation in the enforcement of this agreement.” “Today's agreement is the result of a joint effort between the Department of Justice and the City of Miami to ensure that the Miami Police Department continues its efforts to make our community safe while protecting the sacred Constitutional rights of all of our citizens,” said U.S. Attorney Ferrer. “Through oversight and communication, the agreement seeks to make permanent the positive changes that former Chief Orosa and Chief Llanes have made, and we applaud the City Commission’s vote.” The settlement agreement builds upon important reforms implemented by the city since the Justice Department issued its findings, including: The investigation was conducted by attorneys and staff from the Civil Rights Division’s Special Litigation Section and the Civil Division of the U. S. Attorney’s Office of the Southern District of Florida.",reach comprehens settlement agreement citi miami miami polic resolv shoot announc princip deputi general vanita head wifredo ferrer southern approv citi commiss today effect agreement sign resolv claim stem shoot conduct violent crime control enforc issu juli identifi pattern practic excess forc shoot violat fourth amend complianc settlement monitor independ former polic chief jane settlement citi implement comprehens reform ensur constitut polic support public settlement agreement design minim shoot effect quick investig shoot measur settlement repres renew commit citi miami chief rodolfo llane provid constitut polic miami resid protect public safeti sustain said princip deputi general agreement help strengthen relationship communiti serv improv account offic fire weapon provid communiti particip enforc agreement result joint effort citi miami ensur miami polic continu effort make communiti safe protect sacr constitut said oversight agreement seek make perman posit chang former chief orosa chief llane applaud citi settlement agreement build upon import reform implement citi sinc issu conduct attorney staff special litig section southern


## 2.2 Create a document-term matrix from the preprocessed press releases and to explore top words (5 points)

A. Use the `create_dtm` function I provide (alternately, feel free to write your own!) and create a document-term matrix using the preprocessed press releases; make sure metadata contains the following columns: `id`, `compound` sentiment column you added, and the `topics_clean` column

B. Print the top 10 words for press releases with compound sentiment in the top 5% (so the most positive sentiment)

C. Print the top 10 words for press releases with compound sentiment in the bottom 5% (so the most negative sentiment)

**Hint**: for these, remember the pandas quantile function from pset two.  

D. Print the top 10 words for press releases in each of the three `topics_clean`

For steps B - D, to receive full credit, write a function `get_topwords` that helps you avoid duplicated code when you find top words for the different subsets of the data. There are different ways to structure it but one way is to feed it subsetted data (so data subsetted to one topic etc.) and for it to get the top words for that subset.

**Resources**:

- Here contains an example of applying the create_dtm function: 
https://github.com/rebeccajohnson88/PPOL564_slides_activities/blob/main/activities/fall_22/solutions/09_textasdata_partII_topicmodeling_solution.ipynb


In [18]:
def create_dtm(list_of_strings, metadata):
    vectorizer = CountVectorizer(lowercase = True)
    dtm_sparse = vectorizer.fit_transform(list_of_strings)
    dtm_dense_named = pd.DataFrame(dtm_sparse.todense(), 
        columns=vectorizer.get_feature_names())
    dtm_dense_named_withid = pd.concat([metadata.reset_index(), dtm_dense_named], axis = 1)
    return(dtm_dense_named_withid)

In [19]:
## your code 

#A
#Use the create_dtm function and create a document-term matrix using the preprocessed press releases; 
#make sure metadata contains the following columns: id, compound sentiment column you added, and the topics_clean column
doj_subset_wscores_meta = doj_subset_wscores_proc.add_suffix("_meta").copy()
dtm_doj= create_dtm(doj_subset_wscores_meta.processed_text_meta, doj_subset_wscores_meta[['id_meta', 'compound_meta', 'topics_clean_meta']])

#Write a function that helps to get the top 10 words based on the condition
def get_topwords(subset):
    terms=subset[[col for col in subset.columns
                      if "_meta" not in col and
                      col != "index"]].sum(axis = 0)
    return terms.sort_values(ascending = False)[:10]


#B. Print the top 10 words for press releases with compound sentiment in the top 5% (so the most positive sentiment)

#subset for the top 5% of the compound sentiment
top_compound = dtm_doj.sort_values("compound_meta", ascending = False)[:round(dtm_doj.shape[0]*0.05)]

#print the top 10 words for press releases from the subset
print("the top 10 words for press releases with compound sentiment in the top 5%")
get_topwords(top_compound)

#C. Print the top 10 words for press releases with compound sentiment in the bottom 5% 

#subset for the bottom 5% of the compound sentiment
bottom_compound = dtm_doj.sort_values("compound_meta", ascending = False)[round(dtm_doj.shape[0]*0.95):]

#print the top 10 words for press releases from the subset
print("the top 10 words for press releases with compound sentiment in the bottom 5%")
get_topwords(bottom_compound)

#D. Print the top 10 words for press releases in each of the three `topics_clean`

#subsetting based on topics
subset_cr = dtm_doj[dtm_doj.topics_clean_meta == "Civil Rights"]
subset_psc = dtm_doj[dtm_doj.topics_clean_meta == "Project Safe Childhood"]
subset_hc = dtm_doj[dtm_doj.topics_clean_meta == "Hate Crimes"]

#print the top 10 words for civil right topic
print("the top 10 words for press releases that are on Civil Right")
get_topwords(subset_cr)

#print the top 10 words for Project Safe Childhood
print("the top 10 words for press releases that are on Project Safe Childhood")
get_topwords(subset_psc)

#print the top 10 words for Hate Crime
print("the top 10 words for press releases that are on Hate Crime")
get_topwords(subset_hc)




the top 10 words for press releases with compound sentiment in the top 5%


agreement     140
enforc        112
ensur         108
state          92
court          78
settlement     75
said           72
provid         71
general        71
servic         71
dtype: int64

the top 10 words for press releases with compound sentiment in the bottom 5%


crime       132
assault     122
hate        114
defend      111
sentenc      88
offic        87
guilti       87
prosecut     84
victim       83
feder        80
dtype: int64

the top 10 words for press releases that are on Civil Right


hous         588
offic        503
enforc       498
said         488
discrimin    474
feder        459
violat       444
alleg        389
general      386
state        347
dtype: int64

the top 10 words for press releases that are on Project Safe Childhood


child       994
exploit     673
sexual      570
safe        475
project     472
crimin      403
prosecut    357
sentenc     325
children    313
investig    254
dtype: int64

the top 10 words for press releases that are on Hate Crime


prosecut    458
crime       443
sentenc     420
hate        419
feder       416
charg       403
said        401
defend      396
guilti      373
year        306
dtype: int64

## 2.3 Estimate a topic model using those preprocessed words (5 points)

A. Going back to the preprocessed words from part 2.1, estimate a topic model with 3 topics, since you want to see if the unsupervised topic models recover different themes for each of the three manually-labeled topics (civil rights; hate crimes; project safe childhood). You have free rein over the other topic model parameters beyond the number of topics.

B. After estimating the topic model, print the top 15 words in each topic.

**Hints and Resources**:

- Same topic modeling resources linked to above
- Make sure to use the `random_state` argument within the model so that the numbering of topics does not move around between runs of your code

In [20]:
## your code
#word_tokenize and tokenize automatic way of dividing
text_raw_tokens =[wordpunct_tokenize(one_text) for one_text in doj_subset_wscores_proc.processed_text]

#use gensim to create dictionary
text_raw_dict = corpora.Dictionary(text_raw_tokens)

#shows the mapping of the word from the dictionary
corpus_fromdict = [text_raw_dict.doc2bow(one_text) for one_text in text_raw_tokens]

ldamod = gensim.models.ldamodel.LdaModel (corpus_fromdict,
                                          num_topics= 3, id2word = text_raw_dict,
                                          passes= 6, alpha ='auto',
                                            per_word_topics = True, random_state= 12345
                                          )

#print the topics
topics = ldamod.print_topics(num_words = 15)
for topic in topics:
    print(topic)

(0, '0.011*"prosecut" + 0.011*"sentenc" + 0.011*"charg" + 0.010*"feder" + 0.009*"guilti" + 0.009*"said" + 0.009*"defend" + 0.008*"crime" + 0.008*"investig" + 0.008*"indict" + 0.008*"year" + 0.007*"hate" + 0.007*"victim" + 0.007*"plead" + 0.006*"general"')
(1, '0.013*"hous" + 0.011*"discrimin" + 0.008*"enforc" + 0.008*"said" + 0.007*"disabl" + 0.007*"alleg" + 0.007*"fair" + 0.006*"agreement" + 0.006*"general" + 0.006*"court" + 0.006*"violat" + 0.006*"ensur" + 0.006*"inform" + 0.006*"requir" + 0.006*"file"')
(2, '0.026*"child" + 0.017*"exploit" + 0.015*"sexual" + 0.012*"safe" + 0.012*"project" + 0.010*"crimin" + 0.010*"prosecut" + 0.009*"offic" + 0.008*"sentenc" + 0.008*"children" + 0.008*"state" + 0.007*"investig" + 0.006*"guilti" + 0.006*"general" + 0.006*"engag"')


## 2.4 Add topics back to main data and explore correlation between manual labels and our estimated topics (10 points)

A. Extract the document-level topic probabilities. Within `get_document_topics`, use the argument `minimum_probability` = 0 to make sure all 3 topic probabilities are returned. Write an assert statement to make sure the length of the list is equal to the number of rows in the `doj_subset_wscores` dataframe

B. Add the topic probabilities to the `doj_subset_wscores` dataframe as columns and create a column, `top_topic`, that reflects each document to its highest-probability topic (eg topic 1, 2, or 3)

C. For each of the manual labels in `topics_clean` (Hate Crime, Civil Rights, Project Safe Childhood), print the breakdown of the % of documents with each top topic (so, for instance, Hate Crime has 246 documents-- if 123 of those documents are coded to topic_1, that would be 50%; and so on). 
**Hint**:    
pd.crosstab and normalize may be helpful: https://pandas.pydata.org/pandas-docs/version/0.23/generated/pandas.crosstab.html

D. Using a couple press releases as examples, write a 1-2 sentence interpretation of why some of the manual topics map on more cleanly to an estimated topic than other manual topic(s)

**Resources**:

- End of this code (`Additional summaries of topics and documents`) contains example of how to use `get_document_topics` and other steps to add topic probabilities back to data: 
https://github.com/rebeccajohnson88/PPOL564_slides_activities/blob/main/activities/fall_22/solutions/09_textasdata_partII_topicmodeling_solution.ipynb
- If you're getting errors, use shape, len, and other commands to check the dimensionality of things at different steps since documents may be dropped if they contain no words post-processing 

In [21]:
## your code here to get doc-level topic probabilities 

#extract the document-level probability
l =[ldamod.get_document_topics(item, minimum_probability=0) for item in corpus_fromdict]
assert len(l) == doj_subset_wscores_proc.shape[0]


In [22]:
## your code here to add those topic probabilities to the dataframe
topic_prob = pd.concat([pd.DataFrame(l[index], columns = ['topic', 'possibility']).set_index('topic').transpose() \
           for index in range(len(l))]).reset_index()
topic_prob = topic_prob[[0,1,2]]

#Add the topic probabilities to the doj_subset_wscores dataframe as columns
doj_subset_wscore = doj_subset_wscore.join(topic_prob)

#create a column, top_topic, that reflects each document to its highest-probability topic (eg topic 1, 2, or 3)
doj_subset_wscore['top_topic'] = [doj_subset_wscore[[0,1,2]].iloc[index].idxmax() for index in range(doj_subset_wscore.shape[0])]


In [23]:
## your code here to summarize the topic proportions for each of the topics_clean 
crosstab_doj = pd.crosstab(doj_subset_wscore['topics_clean'], doj_subset_wscore['top_topic'])

doj_pct = pd.DataFrame([crosstab_doj.iloc[index]/crosstab_doj.iloc[index].sum() for index in range(crosstab_doj.shape[0])])
doj_pct

top_topic,0,1,2
Civil Rights,0.327869,0.652459,0.019672
Hate Crimes,0.995935,0.000000,0.004065
Project Safe Childhood,0.012048,0.000000,0.987952


D. Using a couple press releases as examples, write a 1-2 sentence interpretation of why some of the manual topics map on more cleanly to an estimated topic than other manual topic(s)


A former supervisory correctional officer at Louisiana State Penitentiary in Angola, Louisiana, pleaded guilty yesterday in connection with the beating of a handcuffed and shackled inmate, in addition to conspiring to cover up their misconduct by falsifying official records and lying to internal investigators about what happened.    James Savoy, 39, of Marksville, Louisiana, admitted during his plea hearing that he witnessed other officers using excessive force against the inmate and failed to intervene; that he conspired with other officers to cover up the beating by engaging in a variety of obstructive acts; and that he personally falsified official prison records to cover up the attack.   Scotty Kennedy, 48, of Beebe, Arkansas, and John Sanders, 30, of Marksville, Louisiana previously pleaded guilty in November 2016, and September 2017, for their roles in the beating and cover up.   “Every citizen has the right to due process and protection from unreasonable force, and correctional officers who violate these basic Constitutional rights must be held accountable for their egregious actions” said Acting Assistant Attorney General John Gore of the Civil Rights Division.  “The Justice Department will continue to vigorously prosecute correctional officers who violate the public’s trust by committing crimes and to covering up violations of federal criminal law.”   “Yesterday is another example of our office’s unwavering commitment to pursuing those who violate the federal criminal civil rights laws,” said Acting United States Attorney for the Middle District of Louisiana Corey Amundson. “We will continue to work closely with the Justice Department’s Civil Rights Division and the FBI to ensure that no one is above the law.”    This case is being investigated by the FBI’s Baton Rouge Resident Agency and is being prosecuted by Assistant U.S. Attorney Frederick A. Menner, Jr. of the Middle District of Louisiana and Trial Attorney Christopher J. Perras of the Civil Rights Division’s Criminal Section.	


On, Feb. 27, 2017, Michael John Harris, 34, was sentenced to six months imprisonment and one year of supervised release for making telephonic threats to two medical clinics in Minneapolis, Minnesota, that provide reproductive health services.     On March 2, 2016, Harris pleaded guilty to two violations of 18 U.S.C. § 248(a)(1).  During his plea hearing, Harris admitted that on May 12, 2014, he made telephonic threats to two different health clinics in Minneapolis that provide reproductive health services.  In a call to the first clinic, Harris threatened to kill the recipient of the call with his bare hands and to cut the recipient’s head off with a band saw.  In a call to the second clinic, Harris told the recipient that he was going to kill the recipient and the recipient’s co-workers, and that he was going to travel to the clinic and shoot everyone present.  Harris further admitted that he made these threats because the recipient was and has been, and in order to intimidate the recipient and any other person from, obtaining and providing reproductive health services.  “This defendant threatened these clinic workers with death and brutality,” said Acting Assistant Attorney General Tom Wheeler of the Justice Department’s Civil Rights Division. “The department is pleased that the defendant accepted responsibility and will face consequences for his actions. The Department is committed to vigorously enforcing the civil rights of all individuals in this country.” “The violence threatened by this defendant against health care workers is unacceptable,” said United States Attorney Andrew M. Luger of the District of Minnesota.  “This sentence should serve as a reminder to individuals who would engage in such threats that the federal government will prosecute these crimes.” This case is being investigated by the Federal Bureau of Investigation, and is being prosecuted by Trial Attorney Risa Berkower of the Civil Rights Division of the United States Department of Justice and Assistant U.S. Attorney Manda M. Sertich of the U.S. Attorney’s Office for the District of Minnesota.

We think that the contents on Civil Rights are not cleanly mapped on estimated topics because computer perceives topic one differently from how human beings perceive civil rights. Human beings would concentrate on the subject of the attack, but the computer counts the occurances of common words in the topic. Physical attack, shown in the above examples, can happen to an individual based on identity or by occupation or status, but they will both share similar words because the execution of the crime was similar.

In [24]:
doj_subset_wscore[(doj_subset_wscore.top_topic == 0) & (doj_subset_wscore.topics_clean == 'Civil Rights')]

,index,id,title,contents,date,topics_clean,components_clean,neg,neu,pos,compound,0,1,2,top_topic
0,77,17-1235,Additional Former Correctional Officer Pleads Guilty to Beating of Handcuffed and Shackled Inmate at Angola State Prison,"A former supervisory correctional officer at Louisiana State Penitentiary in Angola, Louisiana, pleaded guilty yesterday in connection with the beating of a handcuffed and shackled inmate, in addition to conspiring to cover up their misconduct by falsifying official records and lying to internal investigators about what happened. James Savoy, 39, of Marksville, Louisiana, admitted during his plea hearing that he witnessed other officers using excessive force against the inmate and failed to intervene; that he conspired with other officers to cover up the beating by engaging in a variety of obstructive acts; and that he personally falsified official prison records to cover up the attack. Scotty Kennedy, 48, of Beebe, Arkansas, and John Sanders, 30, of Marksville, Louisiana previously pleaded guilty in November 2016, and September 2017, for their roles in the beating and cover up. “Every citizen has the right to due process and protection from unreasonable force, and correctional officers who violate these basic Constitutional rights must be held accountable for their egregious actions” said Acting Assistant Attorney General John Gore of the Civil Rights Division. “The Justice Department will continue to vigorously prosecute correctional officers who violate the public’s trust by committing crimes and to covering up violations of federal criminal law.” “Yesterday is another example of our office’s unwavering commitment to pursuing those who violate the federal criminal civil rights laws,” said Acting United States Attorney for the Middle District of Louisiana Corey Amundson. “We will continue to work closely with the Justice Department’s Civil Rights Division and the FBI to ensure that no one is above the law.” This case is being investigated by the FBI’s Baton Rouge Resident Agency and is being prosecuted by Assistant U.S. Attorney Frederick A. Menner, Jr. of the Middle District of Louisiana and Trial Attorney Christopher J. Perras of the Civil Rights Division’s Criminal Section.",2017-11-02T00:00:00-04:00,Civil Rights,"Civil Rights Division; USAO - Louisiana, Middle",0.199,0.752,0.049,-0.9931,0.998835,0.000605,0.000560,0
14,423,17-240,Anoka County Resident Sentenced to Six Months in Prison for Threatening Two Clinics that Provide Reproductive Health Services,"On, Feb. 27, 2017, Michael John Harris, 34, was sentenced to six months imprisonment and one year of supervised release for making telephonic threats to two medical clinics in Minneapolis, Minnesota, that provide reproductive health services. On March 2, 2016, Harris pleaded guilty to two violations of 18 U.S.C. § 248(a)(1). During his plea hearing, Harris admitted that on May 12, 2014, he made telephonic threats to two different health clinics in Minneapolis that provide reproductive health services. In a call to the first clinic, Harris threatened to kill the recipient of the call with his bare hands and to cut the recipient’s head off with a band saw. In a call to the second clinic, Harris told the recipient that he was going to kill the recipient and the recipient’s co-workers, and that he was going to travel to the clinic and shoot everyone present. Harris further admitted that he made these threats because the recipient was and has been, and in order to intimidate the recipient and any other person from, obtaining and providing reproductive health services. “This defendant threatened these clinic workers with death and brutality,” said Acting Assistant Attorney General Tom Wheeler of the Justice Department’s Civil Rights Division. “The department is pleased that the defendant accepted responsibility and will face consequences for his actions. The Department is committed to vigorously enforcing the civil rights of all individuals in this c

# 3. Extend the analysis from unigrams to bigrams (9 points)

In the previous question, you found top words via a unigram representation of the text. Now, we want to see how those top words change with bigrams (pairs of words)

A. Using the `doj_subset_wscore` data and the `processed_text` column (so the words after stemming/other preprocessing), create a column in the data called `processed_text_bigrams` that combines each consecutive pairs of word into a bigram separated by an underscore. Eg:

"depart reach settlem" would become "depart_reach reach_settlem"

Do this by writing a function `create_bigram_onedoc` that takes in a single `processed_text` string and returns a string with its bigrams structured similarly to above example
 
**Hint**: there are many ways to solve but `zip` may be helpful: https://stackoverflow.com/questions/21303224/iterate-over-all-pairs-of-consecutive-items-in-a-list

B. Print the `id`, `processed_text`, and `processed_text_bigram` columns for press release with id = 16-217

In [25]:
## your code here 
#I will use doj_subset_wscores_proc instead because I saved the processed_text in this dataframe

def create_bigram_onedoc(processed_text):
    
    #split the text into words.
    process_piece = word_tokenize(processed_text)
    
    #use zip to create a pair
    pair=[process_piece[turn] + "_"+ process_piece[turn+1] for turn in range(len(process_piece)-1)]
    pair_text = " ".join(pair)
    
    return pair_text

#Apply to the doj_subset_wscores_proc
text_bigrams= pd.DataFrame([create_bigram_onedoc(doj_subset_wscores_proc.processed_text[index]) \
              for index in range(doj_subset_wscores_proc.shape[0])],columns = ["processed_text_bigram"])

#I save it to doj_subset_wscore again to reduce confusion in the future
doj_subset_wscore= doj_subset_wscores_proc.join(text_bigrams)

#B
#Print the id, processed_text, and processed_text_bigram columns for press release with id = 16-217
doj_subset_wscore[doj_subset_wscore.id == '16-217'][['id', 'processed_text', 'processed_text_bigram']]

,id,processed_text,processed_text_bigram
313,16-217,reach comprehens settlement agreement citi miami miami polic resolv shoot announc princip deputi general vanita head wifredo ferrer southern approv citi commiss today effect agreement sign resolv claim stem shoot conduct violent crime control enforc issu juli identifi pattern practic excess forc shoot violat fourth amend complianc settlement monitor independ former polic chief jane settlement citi implement comprehens reform ensur constitut polic support public settlement agreement design minim shoot effect quick investig shoot measur settlement repres renew commit citi miami chief rodolfo llane provid constitut polic miami resid protect public safeti sustain said princip deputi general agreement help strengthen relationship communiti serv improv account offic fire weapon provid communiti particip enforc agreement result joint effort citi miami ensur miami polic continu effort make communiti safe protect sacr constitut said oversight agreement seek make perman posit chang former chief orosa chief llane applaud citi settlement agreement build upon import reform implement citi sinc issu conduct attorney staff special litig section southern,reach_comprehens comprehens_settlement settlement_agreement agreement_citi citi_miami miami_miami miami_polic polic_resolv resolv_shoot shoot_announc announc_princip princip_deputi deputi_general general_vanita vanita_head head_wifredo wifredo_ferrer ferrer_southern southern_approv approv_citi citi_commiss commiss_today today_effect effect_agreement agreement_sign sign_resolv resolv_claim claim_stem stem_shoot shoot_conduct conduct_violent violent_crime crime_control control_enforc enforc_issu issu_juli juli_identifi identifi_pattern pattern_practic practic_excess excess_forc forc_shoot shoot_violat violat_fourth fourth_amend amend_complianc complianc_settlement settlement_monitor monitor_independ independ_former former_polic polic_chief chief_jane jane_settlement settlement_citi citi_implement implement_comprehens comprehens_reform reform_ensur ensur_constitut constitut_polic polic_support support_public public_settlement settlement_agreement agreement_design design_minim minim_shoot shoot_effect effect_quick quick_investig investig_shoot shoot_measur measur_settlement settlement_repres repres_renew renew_commit commit_citi citi_miami miami_chief chief_rodolfo rodolfo_llane llane_provid provid_constitut constitut_polic polic_miami miami_resid resid_protect protect_public public_safeti safeti_sustain sustain_said said_princip princip_deputi deputi_general general_agreement agreement_help help_strengthen strengthen_relationship relationship_communiti communiti_serv serv_improv improv_account account_offic offic_fire fire_weapon weapon_provid provid_communiti communiti_particip particip_enforc enforc_agreement agreement_result result_joint joint_effort effort_citi citi_miami miami_ensur ensur_miami miami_polic polic_continu continu_effort effort_make make_communiti communiti_safe safe_protect protect_sacr sacr_constitut constitut_said said_oversight oversight_agreement agreement_seek seek_make make_perman perman_posit posit_chang chang_former former_chief chief_orosa orosa_chief chief_llane llane_applaud applaud_citi citi_settlement settlement_agreement agreement_build build_upon upon_import import_reform reform_implement implement_citi citi_sinc sinc_issu issu_conduct conduct_attorney attorney_staff staff_special special_litig litig_section section_southern


C. Use the create_dtm function and the `processed_text_bigrams` column to create a document-term matrix (`dtm_bigram`) with these bigrams. Keep the following three columns in the data: `id`, `topics_clean`, and `compound` 

D. Print the 
 (1) dimensions of the `dtm` matrix from question 2.2  and 
 (2) the dimensions of the `dtm_bigram` matrix. Comment on why the bigram matrix has more dimensions than the unigram matrix 

E. Find and print the 10 most prevelant bigrams for each of the three topics_clean using the `get_topwords` function from 2.2


In [26]:
## your code here
#C. Use the create_dtm function and the `processed_text_bigrams` column to create a document-term matrix (`dtm_bigram`) with these bigrams. 
#Keep the following three columns in the data: `id`, `topics_clean`, and `compound` 
doj_subset_wscore = doj_subset_wscore.add_suffix("_meta").copy() 
doj_subset_wscore.columns
dtm_bigram = create_dtm(doj_subset_wscore.processed_text_bigram_meta, doj_subset_wscore[['id_meta', 'topics_clean_meta', 'compound_meta']])

#D. Print the 
# (1) dimensions of the `dtm` matrix from question 2.2  and 
# (2) the dimensions of the `dtm_bigram` matrix. 
# Comment on why the bigram matrix has more dimensions than the unigram matrix 

print('The dimension of the dtm_doj is: ', dtm_doj.shape)

print('The dimension of the dtm_bigram is: ', dtm_bigram.shape)

#E. Find and print the 10 most prevelant bigrams for each of the three topics_clean using the `get_topwords` function from 2.2
subset_cr1 = dtm_bigram[dtm_bigram.topics_clean_meta == "Civil Rights"]
subset_psc1 = dtm_bigram[dtm_bigram.topics_clean_meta == "Project Safe Childhood"]
subset_hc1 = dtm_bigram[dtm_bigram.topics_clean_meta == "Hate Crimes"]

get_topwords(subset_cr1)
get_topwords(subset_psc1)
get_topwords(subset_hc1)



Index(['index_meta', 'id_meta', 'title_meta', 'contents_meta', 'date_meta',
       'topics_clean_meta', 'components_clean_meta', 'neg_meta', 'neu_meta',
       'pos_meta', 'compound_meta', 'processed_text_meta',
       'processed_text_bigram_meta'],
      dtype='object')

The dimension of the dtm_doj is:  (717, 6106)
The dimension of the dtm_bigram is:  (717, 63544)


fair_hous         225
deputi_general    221
princip_deputi    220
vanita_head       200
general_vanita    199
said_princip      185
announc_today     116
unit_state        106
consent_decre      92
act_general        78
dtype: int64

project_safe         472
child_exploit        265
child_pornographi    237
sexual_exploit       218
plead_guilti         188
child_sexual         177
exploit_obscen       175
safe_childhood       161
obscen_section       161
exploit_children     156
dtype: int64

hate_crime       299
plead_guilti     267
special_agent    117
year_prison      104
thoma_general     95
grand_juri        93
said_thoma        90
act_general       85
feder_hate        76
face_maximum      75
dtype: int64

# 4. Optional extra credit 1 (1 point)

You notice that the pharmaceutical kickbacks press release we analyzed in question 1 was for an indictment, and that in the original data, there's not a clear label for whether a press release outlines an indictment (charging someone with a crime), a conviction (convicting them after that charge either via a settlement or trial), or a sentencing (how many years of prison or supervised release a defendant is sentenced to after their conviction).

You want to see if you can identify pairs of press releases where one press release is from one stage (e.g., indictment) and another is from a different stage (e.g., a sentencing).

You decide that one way to approach is to find the pairwise string similarity between each of the processed press releases in `doj_subset`. There are many ways to do this, so Google for some approaches, focusing on ones that work well for entire documents rather than small strings.

Find the top two pairs (so four press releases total)-- do they seem like different stages of the same crime or just press releases covering similar crimes?

In [27]:
## your code here

# 5. Optional extra credit 2 (3 points)

Review the scraping code here: https://github.com/jbencina/dojreleases/blob/master/scraper.py
    
Write code to scrape press releases from more recent years from the DOJ website than the years available in the combined.json and produce a visualization of how top words or themes in the press releases changed across the Trump administration (2016-December 2020) versus Biden administration (January 2021 onwards)- you can do this in a .py file that you submit separately and just read in the data produced by that scraping

In [28]:
## your code here